In [17]:
%%pyspark
df = spark.read.load('abfss://files@datalakexxxxxxx.dfs.core.windows.net/sales/orders/2019.csv', format='csv'
## If header exists uncomment line below
, header=True
)
display(df)

StatementMeta(sparkni9wzxh, 2, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6dece194-8655-4a28-8fe4-4c45276a248b)

# Entrenamiento del modelo con datos de 2019

## 1.- Cargar los datos

In [18]:
# Especificamos los nombres de las columnas manualmente
column_names = ["OrderNumber", "Quantity", "OrderDate", "CustomerName", "CustomerEmail", "Product", "ItemQuantity", "UnitPrice", "Tax"]
# Cargar los datos desde Azure Data Lake Storage o tu contenedor de Synapse
df = spark.read.csv('abfss://files@datalakexxxxxxx.dfs.core.windows.net/sales/orders/2019.csv', header=True, inferSchema=True)

# Asignar los nombres de columnas correctos
df = df.toDF(*column_names)

# Mostrar los primeros datos con las columnas correctas
df.show(5)


StatementMeta(sparkni9wzxh, 2, 18, Finished, Available, Finished)

+-----------+--------+----------+--------------+--------------------+--------------------+------------+---------+--------+
|OrderNumber|Quantity| OrderDate|  CustomerName|       CustomerEmail|             Product|ItemQuantity|UnitPrice|     Tax|
+-----------+--------+----------+--------------+--------------------+--------------------+------------+---------+--------+
|    SO43704|       1|2019-07-01|    Julio Ruiz|julio1@adventure-...|Mountain-100 Blac...|           1|  3374.99|269.9992|
|    SO43705|       1|2019-07-01|     Curtis Lu|curtis9@adventure...|Mountain-100 Silv...|           1|  3399.99|271.9992|
|    SO43700|       1|2019-07-01|  Ruben Prasad|ruben10@adventure...|  Road-650 Black, 62|           1| 699.0982| 55.9279|
|    SO43703|       1|2019-07-01|Albert Alvarez|albert7@adventure...|    Road-150 Red, 62|           1|  3578.27|286.2616|
|    SO43697|       1|2019-07-01|   Cole Watson|cole1@adventure-w...|    Road-150 Red, 62|           1|  3578.27|286.2616|
+-----------+---

## 2.- Limpieza de datos

In [22]:
# Convertir fecha a formato adecuado
df = df.withColumn("OrderDate", df["OrderDate"].cast("date"))

# Crear la columna TotalSales
from pyspark.sql.functions import col, expr

df = df.withColumn("TotalSales", (col("ItemQuantity") * col("UnitPrice")) + col("Tax"))
df.show(5)

StatementMeta(sparkni9wzxh, 2, 22, Finished, Available, Finished)

+-----------+--------+----------+--------------+--------------------+--------------------+------------+---------+--------+----------+
|OrderNumber|Quantity| OrderDate|  CustomerName|       CustomerEmail|             Product|ItemQuantity|UnitPrice|     Tax|TotalSales|
+-----------+--------+----------+--------------+--------------------+--------------------+------------+---------+--------+----------+
|    SO43704|       1|2019-07-01|    Julio Ruiz|julio1@adventure-...|Mountain-100 Blac...|           1|  3374.99|269.9992| 3644.9892|
|    SO43705|       1|2019-07-01|     Curtis Lu|curtis9@adventure...|Mountain-100 Silv...|           1|  3399.99|271.9992| 3671.9892|
|    SO43700|       1|2019-07-01|  Ruben Prasad|ruben10@adventure...|  Road-650 Black, 62|           1| 699.0982| 55.9279|  755.0261|
|    SO43703|       1|2019-07-01|Albert Alvarez|albert7@adventure...|    Road-150 Red, 62|           1|  3578.27|286.2616| 3864.5316|
|    SO43697|       1|2019-07-01|   Cole Watson|cole1@adventur

## 3.- Preprocesamiento de los datos

In [23]:
from pyspark.sql.functions import col, year, month

# Crear la columna de ventas totales (TotalSales)
df = df.withColumn("TotalSales", (col("ItemQuantity") * col("UnitPrice")) + col("Tax"))

# Extraer el año y el mes de la columna OrderDate para futuros análisis
df = df.withColumn("Year", year(col("OrderDate"))).withColumn("Month", month(col("OrderDate")))

# Seleccionar las columnas necesarias para el análisis
df = df.select("ItemQuantity", "UnitPrice", "Year", "Month", "TotalSales")

# Mostrar los primeros resultados después de la transformación
df.show(5)


StatementMeta(sparkni9wzxh, 2, 23, Finished, Available, Finished)

+------------+---------+----+-----+----------+
|ItemQuantity|UnitPrice|Year|Month|TotalSales|
+------------+---------+----+-----+----------+
|           1|  3374.99|2019|    7| 3644.9892|
|           1|  3399.99|2019|    7| 3671.9892|
|           1| 699.0982|2019|    7|  755.0261|
|           1|  3578.27|2019|    7| 3864.5316|
|           1|  3578.27|2019|    7| 3864.5316|
+------------+---------+----+-----+----------+
only showing top 5 rows



## 4.- Entrenar el modelo
Una vez que el dataset esté limpio y listo, entrenaremos un modelo de regresión lineal para predecir las ventas totales. Usaremos ItemQuantity, UnitPrice, y las variables de tiempo como Year y Month para predecir TotalSales.

Acciones:
1. Dividir los datos: En conjunto de entrenamiento y conjunto de prueba.
2. Crear el modelo: Utilizaremos un modelo de regresión lineal.
3. Evaluar el modelo: Calcularemos el RMSE (error cuadrático medio) para medir el rendimiento

In [24]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Crear el vector de características
assembler = VectorAssembler(inputCols=["ItemQuantity", "UnitPrice", "Year", "Month"], outputCol="features")

# Transformar los datos
df = assembler.transform(df)

# Dividir los datos en entrenamiento (80%) y prueba (20%)
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

# Definir el modelo de regresión lineal
lr = LinearRegression(featuresCol="features", labelCol="TotalSales")

# Entrenar el modelo
lr_model = lr.fit(train_df)

# Hacer predicciones con el conjunto de prueba
predictions = lr_model.transform(test_df)

# Evaluar el modelo utilizando RMSE
evaluator = RegressionEvaluator(labelCol="TotalSales", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"RMSE del modelo: {rmse}")

# Mostrar algunas predicciones
predictions.select("features", "TotalSales", "prediction").show(5)


StatementMeta(sparkni9wzxh, 2, 24, Finished, Available, Finished)

RMSE del modelo: 1.0994426980360822e-06
+--------------------+----------+-----------------+
|            features|TotalSales|       prediction|
+--------------------+----------+-----------------+
|[1.0,699.0982,201...|  755.0261|755.0261009221855|
|[1.0,699.0982,201...|  755.0261|755.0261009221855|
|[1.0,699.0982,201...|  755.0261|755.0261009221855|
|[1.0,699.0982,201...|  755.0261|755.0261009221855|
|[1.0,699.0982,201...|  755.0261|755.0261009221855|
+--------------------+----------+-----------------+
only showing top 5 rows



# Prediccion de las ventas de 2020

## Preparar datos reales de 2020

In [27]:
# Cargar los datos reales de 2020
data_2020_path = 'abfss://files@datalakexxxxxxx.dfs.core.windows.net/sales/orders/2020.csv'  # Reemplaza {ruta} con la ruta real de los datos
data_2020 = spark.read.csv(data_2020_path, header=True, inferSchema=True)

# Mostrar una muestra de los datos de 2020
data_2020.show(5)


StatementMeta(sparkni9wzxh, 2, 27, Finished, Available, Finished)

+-------+---+----------+-------------+------------------------------+------------------+---+--------+--------+
|SO45347| 11|2020-01-01|Clarence Raji|clarence35@adventure-works.com|Road-650 Black, 52| 16|699.0982| 55.9279|
+-------+---+----------+-------------+------------------------------+------------------+---+--------+--------+
|SO45345|  1|2020-01-01|  Bonnie Yuan|          bonnie12@adventur...|  Road-150 Red, 52|  1| 3578.27|286.2616|
|SO45348|  1|2020-01-01|     Leah Guo|          leah14@adventure-...|  Road-150 Red, 44|  1| 3578.27|286.2616|
|SO45349|  1|2020-01-01|  Candice Sun|          candice19@adventu...|  Road-150 Red, 48|  1| 3578.27|286.2616|
|SO45350|  1|2020-01-01| Ruben Garcia|          ruben16@adventure...|  Road-150 Red, 44|  1| 3578.27|286.2616|
|SO45346|  1|2020-01-01| Dylan Harris|          dylan43@adventure...|  Road-150 Red, 48|  1| 3578.27|286.2616|
+-------+---+----------+-------------+------------------------------+------------------+---+--------+--------+
o

## Transformar datos reales para predicción

Agregar Columnas de Año , Mes y TotalSales 


In [44]:
from pyspark.sql.functions import year, month
from pyspark.sql.functions import col

# Agregar columnas de Year y Month al DataFrame
data_2020 = data_2020.withColumn("Year", year(col("OrderDate"))).withColumn("Month", month(col("OrderDate")))
data_2020 = data_2020.withColumn("TotalSales", (col("ItemQuantity") * col("UnitPrice")) + col("Tax"))
# Mostrar una muestra de los datos después de agregar las columnas
data_2020.show(5)


StatementMeta(sparkni9wzxh, 2, 44, Finished, Available, Finished)

+-----------+--------+----------+------------+--------------------+----------------+------------+---------+--------+----+-----+----------+
|OrderNumber|Quantity| OrderDate|CustomerName|       CustomerEmail|         Product|ItemQuantity|UnitPrice|     Tax|Year|Month|TotalSales|
+-----------+--------+----------+------------+--------------------+----------------+------------+---------+--------+----+-----+----------+
|    SO45345|       1|2020-01-01| Bonnie Yuan|bonnie12@adventur...|Road-150 Red, 52|           1|  3578.27|286.2616|2020|    1| 3864.5316|
|    SO45348|       1|2020-01-01|    Leah Guo|leah14@adventure-...|Road-150 Red, 44|           1|  3578.27|286.2616|2020|    1| 3864.5316|
|    SO45349|       1|2020-01-01| Candice Sun|candice19@adventu...|Road-150 Red, 48|           1|  3578.27|286.2616|2020|    1| 3864.5316|
|    SO45350|       1|2020-01-01|Ruben Garcia|ruben16@adventure...|Road-150 Red, 44|           1|  3578.27|286.2616|2020|    1| 3864.5316|
|    SO45346|       1|2020-

 Transformar los Datos

In [45]:
# Seleccionar las columnas necesarias y crear el vector de características
assembler = VectorAssembler(inputCols=["ItemQuantity", "UnitPrice", "Year", "Month"], outputCol="features")

# Transformar los datos reales de 2020
data_2020_transformed = assembler.transform(data_2020)


# Mostrar una muestra después de la transformación
data_2020_transformed.show(5)


StatementMeta(sparkni9wzxh, 2, 45, Finished, Available, Finished)

+-----------+--------+----------+------------+--------------------+----------------+------------+---------+--------+----+-----+----------+--------------------+
|OrderNumber|Quantity| OrderDate|CustomerName|       CustomerEmail|         Product|ItemQuantity|UnitPrice|     Tax|Year|Month|TotalSales|            features|
+-----------+--------+----------+------------+--------------------+----------------+------------+---------+--------+----+-----+----------+--------------------+
|    SO45345|       1|2020-01-01| Bonnie Yuan|bonnie12@adventur...|Road-150 Red, 52|           1|  3578.27|286.2616|2020|    1| 3864.5316|[1.0,3578.27,2020...|
|    SO45348|       1|2020-01-01|    Leah Guo|leah14@adventure-...|Road-150 Red, 44|           1|  3578.27|286.2616|2020|    1| 3864.5316|[1.0,3578.27,2020...|
|    SO45349|       1|2020-01-01| Candice Sun|candice19@adventu...|Road-150 Red, 48|           1|  3578.27|286.2616|2020|    1| 3864.5316|[1.0,3578.27,2020...|
|    SO45350|       1|2020-01-01|Ruben G

## 3.- Realizar Predicciones para los Datos de 2020

In [46]:
# Hacer predicciones con los datos reales de 2020
predictions_2020 = lr_model.transform(data_2020_transformed)

# Mostrar algunas de las predicciones junto con los valores reales
predictions_2020.select("features", "TotalSales", "prediction").show(5)


StatementMeta(sparkni9wzxh, 2, 46, Finished, Available, Finished)

+--------------------+----------+------------------+
|            features|TotalSales|        prediction|
+--------------------+----------+------------------+
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
+--------------------+----------+------------------+
only showing top 5 rows



## 4.- Evaluar modelo

In [47]:
# Evaluar el modelo utilizando RMSE para los datos de 2020
evaluator = RegressionEvaluator(labelCol="TotalSales", predictionCol="prediction", metricName="rmse")
rmse_2020 = evaluator.evaluate(predictions_2020)
print(f"RMSE para los datos de 2020: {rmse_2020}")


StatementMeta(sparkni9wzxh, 2, 47, Finished, Available, Finished)

RMSE para los datos de 2020: 2.165269526897281e-05


# Comparacion de resultados

In [53]:
# Transformar los datos de 2020 para incluir la columna 'features'
data_2020_transformed = assembler.transform(data_2020)

# Hacer predicciones con los datos de 2020 utilizando el modelo entrenado con datos de 2019
predictions_2020 = lr_model.transform(data_2020_transformed)

# Mostrar una muestra de las predicciones junto con los valores reales de TotalSales
predictions_2020.select("features", "TotalSales", "prediction").show()

StatementMeta(sparkni9wzxh, 2, 53, Finished, Available, Finished)

+--------------------+----------+------------------+
|            features|TotalSales|        prediction|
+--------------------+----------+------------------+
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,699.0982,202...|  755.0261| 755.0261010375974|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.5315994302973|
|[1.0,3578.27,2020...| 3864.5316|3864.53159943